# Import CAMD-XRD Package - hsg and relevant packages

In [1]:
from tqdm.notebook import tqdm
import hsg
import numpy as np
from pymatgen.analysis.diffraction import xrd
from pymatgen.core import Structure

# Relevant Parameters from PbSO4 Pattern


In [2]:
#Li3CrO4
c= 8.560798; a=5.521837; b= 7.099335; alpha=90; beta=90; gamma=90
atoms = [(4,'Pb'), (16,'O'), (4,'S')]
d_tol = 1.2
d_mins = {}
spgs = [62]
pbso4 = Structure.from_file('PbSO4_AMS_DATA.cif')

xrd_calc = xrd.XRDCalculator()
pbso4twothetas = xrd_calc.get_pattern(pbso4).x
pbso4intensities = xrd_calc.get_pattern(pbso4).y
strucs_new_full = []

for i in range(len(spgs)):
    density = 0.4
    
    hsgtest = hsg.HierarchicalStructureGeneration(spgs[i], a,b,c, alpha, beta, gamma, atoms, d_tol, d_mins, \
                                                         use_asu = True)

    for k in tqdm(range(len(hsgtest.filter_combinations))):
        strucs_new=hsgtest.get_structure_grid(density,combination = k)
        strucs_new_full.append([])
        for struc in tqdm(strucs_new):
            strucs_new_full[-1].append(hsgtest.get_pymatgen_structure(struc))

# Calculate XRD patterns using Pymatgen

In [ ]:
matpatterns_all = []
xrd_calc = xrd.XRDCalculator()
for strucs_spg in tqdm(strucs_new_full):
    patterns = []
    for strucs in strucs_spg:
        patterns.append(xrd_calc.get_pattern(strucs))
    matpatterns_all.append(patterns)


# Calculate R values

In [ ]:
Rvalues = []
for j in tqdm(range(len(matpatterns_all))):
    Rvalues.append(hsg.get_Rs(lifeo2twothetas, lifeo2intensities, matpatterns_all[j]))

# Determine lowest R value structures for DFT relaxation

In [ ]:
lowests = []
struc_indexes = []
lowest = []
for i in range(len(Rvalues)):
    for k in range(len(Rvalues[i])):

        if len(lowest) == 100:
            order = np.argsort(lowest)
            if Rvalues[i][k] < lowest[order[-1]]:
                lowest[order[-1]] = Rvalues[i][k]
                struc_indexes[order[-1]] = (i,k)
        else:
            lowest.append(Rvalues[i][k])
            struc_indexes.append((i,k))

test_strucs = []

for index2 in struc_indexes:
    test_strucs.append(strucs_new_full[index2[0]][index2[1]])
